In [18]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import geopy
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

ModuleNotFoundError: No module named 'geopy'

In [54]:
# hospitals = pd.read_csv("Resources/Clean/San_Diego_Hospital_Data.csv")
schools = pd.read_csv("Resources/Clean/San_Diego_School_Data.csv")
businesses = pd.read_csv("Resources/Clean/San_Diego_Markets.csv")
transit = pd.read_csv("Resources/Clean/San_Diego_Transit.csv")
parks = pd.read_csv("Resources/Clean/San_Diego_Parks.csv")

businesses

,zipcode,business_status,name,lat,lng,country_code
0,91910,OPERATIONAL,Smart & Final Extra!,32.639000,-117.050924,us
1,91910,OPERATIONAL,Sprouts Farmers Market,32.627807,-117.074994,us
2,91911,OPERATIONAL,Carnival Supermarket,32.621859,-117.072892,us
3,91910,OPERATIONAL,Target Grocery,32.653616,-117.065593,us
4,91910,OPERATIONAL,99 Ranch Market,32.629342,-117.040931,us
...,...,...,...,...,...,...
463,92154,OPERATIONAL,El Picador Foods,32.573865,-117.055439,us
464,92173,OPERATIONAL,Liva Distributor,32.568932,-117.064476,us
465,92173,OPERATIONAL,La Bodega Market,32.552844,-117.042370,us
466,92173,OPERATIONAL,Pancho Villa Mercado,32.561686,-117.063112,us


In [46]:
# Schools Geos
schools_above_average = schools[schools['rating'] >= 7]
schools_above_average
school_geos = schools_above_average[['lat','lng','zipcode']]
school_geos['resource'] = 'school'
school_geos = school_geos.set_index('zipcode')
school_geos

/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/4232580000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_geos['resource'] = 'school'


,lat,lng,resource
zipcode,,,
92037,32.811186,-117.263065,school
92093,32.878454,-117.229467,school
92037,32.839236,-117.271699,school
92037,32.859286,-117.242970,school
92037,32.832408,-117.264719,school
...,...,...,...
92024,33.037340,-117.274455,school
92024,33.039846,-117.277131,school
92024,33.057975,-117.298047,school


In [57]:
# Business Geos
business_geos = businesses[['lat','lng','zipcode']]
business_geos['resource'] = 'business'
business_geos = business_geos.set_index('zipcode')
business_geos

/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/211355381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_geos['resource'] = 'business'


,lat,lng,resource
zipcode,,,
91910,32.639000,-117.050924,business
91910,32.627807,-117.074994,business
91911,32.621859,-117.072892,business
91910,32.653616,-117.065593,business
91910,32.629342,-117.040931,business
...,...,...,...
92154,32.573865,-117.055439,business
92173,32.568932,-117.064476,business
92173,32.552844,-117.042370,business


In [56]:
# Transit Geos
transit_geos = transit[['lat','lng','Zip Code']]
transit_geos['resource'] = 'transit'
transit_geos.rename(columns = {'Zip Code':'zipcode'}, inplace = True)
transit_geos = transit_geos.set_index('zipcode')
transit_geos

/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/2641201748.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transit_geos['resource'] = 'transit'
/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/2641201748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transit_geos.rename(columns = {'Zip Code':'zipcode'}, inplace = True)


,lat,lng,resource
zipcode,,,
91941,32.765915,-117.019588,transit
91941,32.767805,-117.015427,transit
91977,32.714228,-117.012800,transit
91941,32.769344,-117.011003,transit
91977,32.709168,-117.009255,transit
...,...,...,...
92008,33.178276,-117.336525,transit
92057,33.254410,-117.298129,transit
92075,32.992937,-117.271067,transit


In [59]:
# Business Geos
park_geos = parks[['lat','lng','postcode']]
park_geos.rename(columns = {'postcode':'zipcode'}, inplace = True)
park_geos['resource'] = 'park'
park_geos = park_geos.set_index('zipcode')
park_geos

/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/1051905917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park_geos.rename(columns = {'postcode':'zipcode'}, inplace = True)
/var/folders/jg/35qqdf9j6v74xwfm65sq9zdh0000gn/T/ipykernel_18093/1051905917.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park_geos['resource'] = 'park'


,lat,lng,resource
zipcode,,,
91901,32.819494,-116.763489,park
91901,32.842437,-116.704128,park
91901,32.799750,-116.760434,park
91902,32.682508,-117.002547,park
91902,32.682592,-117.001904,park
...,...,...,...
92763,33.395758,-117.591568,park
92051,33.402802,-117.551148,park
92763,33.389572,-117.593152,park


In [ ]:
clean_geos_all = pd.concat([school_geos, business_geos, transit_geos, park_geos])
clean_geos_all